# Lesson 23: Random number generation

<hr>

In [1]:
import numpy as np
import pandas as pd
import scipy.stats

import bokeh_catplot

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

<hr>

**Random number generation** (RNG), besides being a [song in the original off-Broadway run of *Hedwig and the Angry Inch*](https://www.youtube.com/watch?v=KzGHjsMGdy0), is the process by which a string of random numbers may be drawn. Of course, the numbers are not completely random for several reasons.

1. They are drawn from a probability distribution. The most common one is the uniform distribution on the domain $0 \le x < 1$, i.e., random numbers between zero and one. ("Completely random" does not make sense because of the infinite magnitude of numbers.) 
2. In most computer applications, including the ones we'll use in bootcamp, the random numbers are actually pseudorandom. They depend entirely on an input **seed** and are then generated by a deterministic algorithm from that seed.

This is a bit academic. Let's jump right in generating random numbers. Much of the random number generation functionality you will need is in the `np.random` module. Let's start by generating random numbers from a Uniform distribution.

In [2]:
np.random.uniform(low=0, high=1, size=10)

array([0.42816061, 0.46028402, 0.12273942, 0.10050953, 0.97443706,
       0.27413244, 0.92991827, 0.10625903, 0.26182781, 0.23487994])

The function `uniform()` in the `np.random` module generates random numbers on the interval \[`low`, `high`) from a Uniform distribution. The `size` kwarg is how many random numbers you wish to generate, and is a kwarg in all of Numpy's random number generators. The random numbers are returned as a NumPy array.

We can check to make sure it is appropriately drawing random numbers out of the uniform distribution by plotting the cumulative distribution function, just like we did last time.  We'll generate 1,000 random numbers and plot them along with the CDF of a Uniform distribution.

In [3]:
# Generate random numbers
x = np.random.uniform(low=0, high=1, size=1000)

# Plot the ECDF of randomly generated numbers
p = bokeh_catplot.ecdf(x)

p.line(
    x=[0, 1],
    y=[0, 1],
    line_width=2,
    line_color='orange',
)

bokeh.io.show(p)

So, it looks like our random number generator is doing a good job.

Generating random numbers on the uniform interval is one of the most commonly used RNG techniques. In fact, many of the other contexts of RNG are derived from draws from the uniform distribution. For example, you can simulate flipping a biased (unfair) coin.

In [4]:
# Generate 20 random numbers on uniform interval
x = np.random.uniform(low=0, high=1, size=20)

# Make the coin flips (< 0.7 means we have a 70% chance of heads)
heads = x < 0.7

# Show which were heads, and count the number of heads
print(heads)
print('\nThere were', np.sum(heads), 'heads.')

[ True  True  True  True False False False False False  True False  True
  True  True  True False False  True  True  True]

There were 12 heads.


## Choice of generator

As of version 1.17.3 of Numpy, the algorithm under the hood of calls to functions like `np.random.uniform()` is the [Mersenne Twister Algorithm](https://en.wikipedia.org/wiki/Mersenne_Twister) for generating random numbers. It is a very widely used and reliable method for generating random numbers. However, starting with version 1.17.4, the `numpy.random` module offers random number generators with better statistical performance, including the [PCG64 generator](http://www.pcg-random.org). Going forward, the preferred approach to doing random number generation is to first instantiate a generator of your choice, and then use its methods to generate numbers out of probability distributions.

Let's set up a PCG64 generator, which is Numpy's default.

In [5]:
rg = np.random.default_rng()

Now that we have the generator, we can use it to draw numbers out of distributions. The syntax is the same as before, except `rg` replaces `np.random`.

In [6]:
rg.uniform(low=0, high=1, size=20)

array([0.39313936, 0.98287716, 0.08655973, 0.76565162, 0.20259894,
       0.5020257 , 0.53493208, 0.12603032, 0.76314581, 0.84669399,
       0.12926163, 0.35004382, 0.96208211, 0.59216024, 0.37498657,
       0.31451715, 0.63471778, 0.25925974, 0.83188035, 0.06018767])

## Seeding random number generators

Now, just to demonstrate that random number generation is deterministic, we will explicitly **seed** the random number generator (which is usually seeded with a number representing the date/time to avoid repeats) to show that we get the same random numbers.

In [7]:
# Instantiate generator with a seed
rg = np.random.default_rng(seed=3252)

# Generate random numbers
rg.uniform(size=10)

array([0.18866535, 0.04418857, 0.02961285, 0.22083971, 0.43341773,
       0.13166813, 0.42112164, 0.43507845, 0.61380912, 0.30627603])

In [8]:
# Re-seed the RNG
rg = np.random.default_rng(seed=3252)

# Generate random numbers
rg.uniform(size=10)

array([0.18866535, 0.04418857, 0.02961285, 0.22083971, 0.43341773,
       0.13166813, 0.42112164, 0.43507845, 0.61380912, 0.30627603])

The random number sequence is exactly the same. If we choose a different seed, we get totally different random numbers.

In [9]:
# Fit one more fan in the 3252
rg = np.random.default_rng(seed=3253)
rg.uniform(size=10)

array([0.31390226, 0.73012457, 0.05800998, 0.01557021, 0.29825701,
       0.10106784, 0.06329107, 0.58614237, 0.52023168, 0.52779988])

If you are writing tests, it is often useful to seed the random number generator to get reproducible results.

## Drawing random numbers out of other distributions

We can also draw random numbers from other probability distributions. For example, say we wanted to draw random samples from a Normal distribution with mean μ and standard deviation σ. (We already saw this example when we were looking at histograms, but we repeat it here.)

In [10]:
# Set parameters
mu = 10
sigma = 1

# Draw 100000 random samples
x = rg.normal(mu, sigma, size=100000)

# Plot the histogram
p = bokeh_catplot.histogram(
    x,
    density=True,
    y_axis_label='approximate PDF',
)

bokeh.io.show(p)

It looks Normal, but, again, comparing the resulting ECDF is a better way to look at this. We'll check out the ECDF with 1000 samples so as not to choke the browser. I will also make use of the theoretical CDF for the Normal distribution available from the `scipy.stats` module.

In [11]:
# Compute theoretical CDF
x_theor = np.linspace(6, 14, 400)
y_theor = scipy.stats.norm.cdf(x_theor, mu, sigma)

# Plot the ECDF of randomly generated numbers
p = bokeh_catplot.ecdf(x[:1000])

p.line(
    x=x_theor,
    y=y_theor,
    line_width=2,
    line_color='orange',
)

bokeh.io.show(p)

Yup, right on!

## Selections from discrete distributions

The random numbers we have generated so far from from continuous probability distributions. We can also draw random numbers from discrete distributions. We already showed how we can do this for "coin flips," but we can do it for other distributions as well. Say we wanted to draw from a Binomial distribution. We can use `rg.binomial()`.

In [12]:
# Draw how many coin flips land heads in 10 files
rg.binomial(10, 0.5)

7

There are other discrete distributions we can draw from, such as Binomial, Geometric, Poisson, etc., and [the documentation](http://docs.scipy.org/doc/numpy/reference/routines.random.html) describes how to use them.

## Choosing elements from an array

It is often useful to randomly choose elements from an existing array. The `rg.choice()` function does this. You equivalently could do this using `rg.integers()`, where the integers represent indices in the array, *except* `rg.choice()` has a great keyword argument, `replace`, which allows random draws with or without replacement. For example, say you had 100 samples that you wanted to send to a facility for analysis, but you can only afford to send 20. If we used `rg.integers()`, we might have a problem.

In [13]:
rg = np.random.default_rng(seed=3252)
np.sort(rg.integers(0, 51, size=20))

array([ 1,  2,  2,  6,  7,  9, 10, 11, 11, 13, 15, 17, 21, 22, 22, 23, 31,
       33, 37, 38])

Samples 2, 11, and 22 we each selected twice!

In [14]:
rg.choice(np.arange(51), size=20, replace=False)

array([ 5, 37, 43, 11, 20, 10, 29, 22, 39, 23, 17,  1, 12, 21, 42, 27, 28,
       36, 25, 31])

Now, because we chose `replace=False`, we do not get any repeats.

### Generating random sequences

Because it works with selecting characters as well as numbers, we can use the `rg.choice()` function to generate random DNA sequences.

In [15]:
''.join(rg.choice(list('ATGC'), replace=True, size=70))

'CAGGAGTCGCTGGGATAACAATTGTGACCTATGTAACTCAGCGAAGAGACTCGGGCCCGACCCACTAAAG'

## Shuffling an array

Similarly, the `np.random.permutation()` function is useful. It takes the entries in an array and shuffles them! Let's shuffle a deck of cards.

In [16]:
rg.permutation(np.arange(53))

array([21, 32, 52, 28, 29, 46,  2,  9, 16, 41, 43, 33, 36, 42, 19, 44, 25,
       50,  0, 27, 35, 24, 26, 30, 10, 11,  5, 14, 23, 17, 40, 12, 13, 49,
       22,  8, 47,  7, 48, 45, 31,  6, 34, 51,  1, 18, 20, 15, 39,  4, 38,
        3, 37])

## When do we need RNG?

Answer: VERY OFTEN!  We will see many examples in the next lessons and in the exercises.  

In many ways, probability is the language of biology. Molecular processes have energetics that are comparable to the thermal energy, which means they are always influenced by random thermal forces. The processes of the central dogma, including DNA replication, are no exceptions. This gives rise to random mutations, which are central to understanding how evolution works. If we want to understand them, it is often useful to use random number generators to model the processes.

RNG also comes up A LOT in data analysis, which we will see in the lessons on hacker stats.

## Computing environment

In [17]:
%load_ext watermark
%watermark -v -p numpy,scipy,pandas,bokeh,bokeh_catplot,jupyterlab

CPython 3.7.7
IPython 7.13.0

numpy 1.18.1
scipy 1.4.1
pandas 0.24.2
bokeh 2.0.2
bokeh_catplot 0.1.8
jupyterlab 1.2.6
